# **Crear un modelo de recomendación de recetas**
Se entrenará un modelo de NearestNeighbors de Scikit-Learn para encontrar las recetas más cercanas a los ingredientes.

## Extraer datos

In [1]:
#importar librerías
import numpy as numpy
import pandas as pd
import sys
import os

#cargar el mlb_ml que se usó para pasar los ingredientes a columnas 
import joblib
mlb = joblib.load("../../limpieza/mlb_ml.pkl")

#importar StandarScaler para escalar los datos del dataset
from sklearn.preprocessing import StandardScaler

#modelo de KNN para encontrar las recetas más cercanas a los ingredientes
from sklearn.neighbors import NearestNeighbors


In [2]:
#extraer "recetas_limpio.csv" para buscar las recetas recomendadas y "recetas_ML.csv" para entrenar el modelo
df_sin_transformar = pd.read_csv('../../limpieza/recetas_limpio.csv')
df_entrenar = pd.read_csv('../../limpieza/recetas_ML.csv')

## Explorar datos

In [3]:
df_sin_transformar.head()

,Nombre,Ingredientes,Categoria
0,Tostadas con mermelada,Pan-Mermelada,Desayuno
1,Facturas de hojaldre,Harina-Manteca-Azucar,Desayuno
2,Tortilla de papas,Papas-Huevos-Cebolla,Almuerzo
3,Milanesa napolitana,Carne-Tomate-Queso,Almuerzo
4,Empanadas de carne,Carne picada-Cebolla-Masa,Almuerzo


In [4]:
df_entrenar.head()

,Aceite,Aceite de oliva,Aceitunas,Aderezo,Agua,Ajo,Albahaca,Albondigas,Algas,Almendras,...,Vainillas,Verduras,Vino blanco,Vino tinto,Yogur,Zanahoria,Zapallitos,Zapallo,Nombre,Categoria
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Tostadas con mermelada,Desayuno
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Facturas de hojaldre,Desayuno
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Tortilla de papas,Almuerzo
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Milanesa napolitana,Almuerzo
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,Empanadas de carne,Almuerzo


In [5]:
#confirmar el tamaño del dataset de entrenamiento: (232, 131)
df_entrenar.shape

(232, 131)

## Escalar datos

In [6]:
#escalar todas las columnas excepto 'Categoria' y 'Nombre' con StandarScaler
X = df_entrenar.drop(['Nombre', 'Categoria'], axis=1)

scaler = StandardScaler()
x_scaler = scaler.fit_transform(X)
x_scaler

array([[-0.16293763, -0.06579517, -0.06579517, ..., -0.13245324,
        -0.06579517, -0.06579517],
       [-0.16293763, -0.06579517, -0.06579517, ..., -0.13245324,
        -0.06579517, -0.06579517],
       [-0.16293763, -0.06579517, -0.06579517, ..., -0.13245324,
        -0.06579517, -0.06579517],
       ...,
       [-0.16293763, -0.06579517, -0.06579517, ..., -0.13245324,
        -0.06579517, -0.06579517],
       [-0.16293763, -0.06579517, -0.06579517, ..., -0.13245324,
        -0.06579517, -0.06579517],
       [-0.16293763, -0.06579517, -0.06579517, ..., -0.13245324,
        -0.06579517, -0.06579517]])

## Entrenar modelo

In [7]:
#crear modelo y entrenarlo con las metricas de "cosine" o coseno para calcular la similitud
knn = NearestNeighbors(n_neighbors=1, metric="cosine")
knn.fit(x_scaler)

NearestNeighbors(metric='cosine', n_neighbors=1)

## Probar modelo

In [8]:
#función para recomendar recetas en base a ingredientes dados
def recomendar_receta(ingredientes_usuario, mlb, scaler, knn):
    #pasar los ingredientes ingresados a one-hot-encoding con el transformador mlb
    ingredientes_mlb = pd.DataFrame(mlb.transform([ingredientes_usuario]), columns=mlb.classes_)
    #escalar ingredientes
    ingredientes_scaler = scaler.transform(ingredientes_mlb)
    #encontrar la receta más cercana usando el modelo knn
    distancias, indices = knn.kneighbors(ingredientes_scaler)
    #mostrar la receta recomendada
    recetas_recomendadas = df_sin_transformar.iloc[indices[0]]
    print(recetas_recomendadas)

In [9]:
#ejemplo
ingredientes_usuario = ["Manzanas", "Azucar"]

#obtener la receta recomendada
recomendar_receta(ingredientes_usuario, mlb, scaler, knn)

              Nombre                   Ingredientes Categoria
83  Torta de manzana  Manzanas-Harina-Huevos-Azucar    Postre


In [10]:
ingredientes_usuario = ["Leche", "Huevos", "Azucar"]

#obtener la receta recomendada
recomendar_receta(ingredientes_usuario, mlb, scaler, knn)

        Nombre         Ingredientes Categoria
9  Flan casero  Huevos-Leche-Azucar    Postre


In [11]:
ingredientes_usuario = ['Leche', 'Huevos', 'Coco rallado']

#obtener la receta recomendada
recomendar_receta(ingredientes_usuario, mlb,scaler, knn)

          Nombre               Ingredientes Categoria
87  Flan de coco  Huevos-Leche-Coco rallado    Postre


## Guardar modelo

In [12]:
#guardar el modelo entrenado
joblib.dump(knn, 'recomendar_receta.pkl')

#guardar scalar ya entrenado para luego usarlo
joblib.dump(scaler, 'scaler_recomendar.pkl')

['scaler_recomendar.pkl']